In [1]:
from glob import glob
from model import Fusionmodel
import torch
from PIL import Image
import torchvision.transforms as transforms
import time
import os
from torchvision.utils import save_image
from utils import mkdir, CustomVisionDataset, gradient, hist_similar
from torch.utils.data import DataLoader
from tqdm import tqdm

from loss_network import LossNetwork
from pytorch_msssim import ms_ssim, ssim
import torch.nn as nn

import pandas as pd
import matplotlib.pyplot as plt
import sewar

from metrics import mutual_information_2d


In [2]:
def test_model():
    # from channel_fusion import channel_f as channel_fusion
    # from utils import mkdir,Strategy 
    #testing remote code

    # _tensor = transforms.ToTensor()
    # _pil_gray = transforms.ToPILImage()
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"
    device = 'cuda'

    model = Fusionmodel().to(device)
    checkpoint = torch.load('train_model_result/Training_best.pkl')
    model.load_state_dict(checkpoint['weight'])

    root = 'Original_test_image'
    sub1 = 'VIS'
    sub2 = 'IR'    
    dataset = CustomVisionDataset(root, sub1, sub2)
    test_dataloader = DataLoader(dataset, shuffle=False)

    # root = '/storage/locnx/SampleVisDrone/SampleVisDroneTest'
    # sub1 = 'VIS'
    # sub2 = 'IR'    
    # dataset = CustomVisionDataset(root, sub1, sub2)
    # test_dataloader = DataLoader(dataset, shuffle=False)

    # root = '/storage/locnx/SampleVisDrone/SampleVisDroneTest2'
    # sub1 = 'VIS'
    # sub2 = 'IR'    
    # dataset = CustomVisionDataset(root, sub1, sub2)
    # test_dataloader = DataLoader(dataset, shuffle=False)

    save_dir = "New_model/output_final" 
    mkdir(save_dir)

    steps = len(test_dataloader)
    print("step:",steps)

    MSE_fun = nn.MSELoss()
    SSIM_fun = ssim
    MS_SSIM_fun = ms_ssim
    CrossEntropyLoss = nn.CrossEntropyLoss()
    ms_ssim_func2 = sewar.full_ref.msssim

    with torch.no_grad():
        loss_network = LossNetwork()
        loss_network.to(device)
    loss_network.eval()

    
    lost_list = []

    def test(model):

        tqdms = tqdm(range(int(steps)))
        s_time = time.time()
        imgs_T = iter(test_dataloader)

        for index in tqdms:
            imgs = next(imgs_T)

            img_vis = imgs[0].to(device)
            img_ir = imgs[1].to(device)
            vis_name = imgs[2]
            ir_name = imgs[3]

            outimage = model(img_vis,img_ir)
            # outimage = out

            name = 'addition'

            # print("input shape:", img1.shape)
            # print("output shape:", out.shape)
           
            ssim1 = SSIM_fun(img_vis, outimage)
            mse_loss1 = MSE_fun(img_vis,outimage)
            grd_loss1 = MSE_fun(gradient(img_vis), gradient(outimage))
            hist_loss1 = hist_similar(img_vis, outimage.detach()) * 0.001
            std_loss1 = torch.abs(outimage.std() - img_vis.std())
            Entrophy1 = CrossEntropyLoss(img_vis, outimage)
            ms_ssim1 = MS_SSIM_fun(img_vis, outimage)
#             mi1 = mutual_information_2d(img_vis, outimage)


            ssim2 = SSIM_fun(img_ir, outimage)
            mse_loss2 = MSE_fun(img_ir,outimage)
            grd_loss2 = MSE_fun(gradient(img_ir), gradient(outimage))
            hist_loss2 = hist_similar(img_ir, outimage.detach()) * 0.001
            std_loss2 = torch.abs(outimage.std() - img_ir.std())
            Entrophy2 = CrossEntropyLoss(img_ir, outimage)
            ms_ssim2 = MS_SSIM_fun(img_ir, outimage)
#             mi2 = mutual_information_2d(img_ir, outimage)


            ssim = (ssim1 + ssim2)/2
            mse_loss = (mse_loss1 + mse_loss2)/2
            std_loss = (std_loss1 + std_loss2)/2
            grd_loss = (grd_loss1 + grd_loss2)/2
            ent_loss = (Entrophy1 + Entrophy2)/2
            ms_ssim = (ms_ssim1 + ms_ssim2)/2
#             mi = (mi1 + mi2)/2
            
            lost = [vis_name ,ent_loss.item(), ssim.item(), ms_ssim.item(), mse_loss.item(), std_loss.item(), grd_loss.item()]
            lost_list.append(lost)

            e_time = time.time() - s_time
            save_name = save_dir+'/'+name+'/fusion'+str(vis_name)+"-"+str(ir_name)+'.jpg'
            mkdir(save_dir+'/'+name)
            # img_fusion = _pil_gray(out)
#             save_image(outimage, save_name)
#             print("pic:[%d] %.4fs %s"%(index,e_time,save_name))
#             print('stat":[%.5f %.5f %.5f %.5f]' % (msssim_loss, mse_loss, std_loss, grd_loss))
    
    with torch.no_grad():
        test(model)
        
    table = pd.DataFrame(lost_list, columns = ['vis_img', 'entropy', 'ssim', 'ms_ssim', 'mse', 'std' , 'grd'])
    return table
    
if __name__ == '__main__':
    print("test initiated")
    result = test_model()
    print("success testing")

test initiated
step: 21


/home/locnx/anaconda3/envs/pytorche2/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/locnx/anaconda3/envs/pytorche2/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|█████████████████████████████████████████████████████████████| 21/21 [00:02<00:00,  7.70it/s]

success testing


In [3]:
result = result.set_index(['vis_img'])
result1 = result

In [4]:
result1.loc['mean'] = result.mean()
result1.loc['var'] = result.var()
result1.loc['std'] = result.std()
result1.to_csv('test_result.csv')

In [5]:
result1

,entrophy,ssim,ms_ssim,mse,std,grd
vis_img,,,,,,
"(1.png,)",1.139073,0.998597,9.999305e-01,0.010763,0.040603,0.048007
"(10.png,)",1.789143,0.996900,9.997213e-01,0.024365,0.073183,0.086837
"(11.png,)",1.236150,0.997992,9.998748e-01,0.014100,0.028716,0.012682
"(12.png,)",1.030072,0.996963,9.998816e-01,0.023089,0.065363,0.056210
"(13.png,)",1.165637,0.996941,9.997209e-01,0.022597,0.059859,0.075565
"(14.png,)",1.724410,0.997327,9.997901e-01,0.020101,0.071716,0.096394
"(15.png,)",1.864353,0.996480,9.998626e-01,0.027819,0.080070,0.187948
"(16.png,)",1.187200,0.994687,9.996294e-01,0.037356,0.113765,0.035792
"(17.png,)",1.208571,0.994929,9.996247e-01,0.036601,0.119871,0.024444


In [6]:
result1[-3:]

,entrophy,ssim,ms_ssim,mse,std,grd
vis_img,,,,,,
mean,1.410068,0.996761,9.996924e-01,0.023924,0.065751,0.078464
var,0.106080,0.000004,1.619243e-07,0.000179,0.000624,0.002535
std,0.418555,0.207847,2.084506e-01,0.013966,0.027934,0.051680
